In [ ]:
import time
import pandas as pd
import numpy as np

# current clustering
df_clusters=pd.read_csv('cluster_stops1.csv') # this is to read from csv file

arena='Schlachthof'
#depot='Hagen Wendeplatz'
#full distance matrix df
df=pd.read_csv('full_distance_matrix_lueneburg.csv',sep=';')
df.set_index("name", inplace = True)

# 1. DATA PRE-PROCESSING

# list of clusters lst. This we will get directly from Andrey's algorithm
lst=[]
for i in range(15):
    lst_i=[]
    for idx,row in df_clusters.iterrows():
        if row['cluster']==i:
            lst_i.append(row['name'])
    lst.append(lst_i)
#remove the stop with 0 passengers    
lst.remove(lst[0])
#------------------------------------

# functions for data pre-processing: the output from Andrey should be carefully checked 
#OUTPUT of this step is the list of clusters, whose number of passengers is 0. Each cluster ends up with arena

#check if a certain stop contained in a cluster
def stInLst(lst,stop):
    for st in lst:
        if st==stop:
            return True
    return False

#check if a stop appeared in a list of clusters
def checkStop(lstoflst,stop):
    i=0
    for sublst in lstoflst:
        i+=1
        if stInLst(sublst,stop):
            print('sublist nr ',i)
            return True
    return False           

#remove arena from the list
#lst.remove(elmt)
for sublst in lst:
    for stop in [arena]:
        if stInLst(sublst,stop)==True:
            sublst.remove(stop)           

# add arena to each route
for sublst in lst:
    #sublst.insert(0,depot)
    sublst.append(arena)

#--------------------------------------------------------------------------------------


# 2. SA ALGORITHM

# function for SA

# define initial temperature for a cluster
def tmax(cluster):
    T=[]
    for st1 in cluster:
        for st2 in cluster:
            T.append(df.loc[st1,st2])
    return max(T)

#two-opt neighborhood operator
# s is a list of stops
def two_opt(s,i,j):
    s_prime=[]
    if i==0:
        t1=[]
    else:
        t1=s[0:i]
    t2=s[i:(j+1)]
    t2.reverse()
    t3=s[(j+1)::]
    s_prime=t1+t2+t3
    return s_prime

#cost function: distance, lst is a list of clusters
def cost(lst):
    l = 0
    for i in range(len(lst)-1):
        l += df.loc[lst[i]][lst[i+1]]
    #l += df.loc[lst[len(lst)-1],stop] 
    return l

#get the optimal route from a list of two routes
#list_check=[(['a'],1),(['b'],2)]
def opt_sol(list_check):
    if list_check[0][1]>list_check[1][1]:
        return list_check[1]
    else:
        return list_check[0]
    
#lst is list of clusters
def shortPath_SA_stops(lst,P,Tmax, alpha):
    best_sol= [] #save the good iterations with improvement (does not take no_change into account if it does not improve)
    k_no_change=0
    s = lst
    c = cost(lst)
    ntrial = 1
    #T = 35 # initial 30
    T=Tmax
    #alpha = 0.9
    while (ntrial <= N) and (T> T0) and (k_no_change<K):   
        for i in range(0,len(lst)-2):# dummy depots
            for j in range(i+1,len(lst)-1):
                s1=two_opt(s,i,j)
                c1 = cost(s1)
                if c1 < c:
                    s, c = s1, c1
                    best_sol.append((s1,c1))
                else:
                    if np.exp(-(c1 - c)/T) > P:
                        s, c = s1, c1
                        k_no_change=0
                    else:
                        k_no_change+=1
        T = alpha*T
        ntrial += 1
    if len(best_sol)!=0:
        lst_opt=[best_sol[-1], (s,c)]
        result=opt_sol(lst_opt)
    else:
        result=s,c
    return result

def run_SA(lst_clusters,P,alpha,K,T0,N):
    start = time.time()
    routes=[]
    total_route=0
    for sublst in lst:
        Tmax=tmax(sublst)
        result=shortPath_SA_stops(sublst,P,Tmax,alpha)
        total_route+=result[-1]
        routes.append(result)
    end=time.time()
    duration=end-start
    return total_route, routes, duration

# 3. RUNNING SA
P=0.5
N=500
T0=0.1
K=200
alpha=0.95
result_run=run_SA(lst,P,alpha,K,T0,N)
print(result_run[0], result_run[2])